In [1]:
import os

import sys
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\src")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\venv")

path = os.getcwd()

curr_path = path.replace('\\', '/')
head_place = curr_path.find('GIEBN')
main_path = curr_path[:head_place]+'GIEBN'
main_path

'c:/Users/User/PycharmProjects/GIEBN'

In [2]:
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, GradientBoostingRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline, make_union
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor


from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel, f_regression

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, RFE

from auxiliary.auxiliary import construct_by_kmeans

from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import BayesianNetwork
import yodo.yodo
import matplotlib.pyplot as plt
from pgmpy.estimators import PC, BicScore, K2Score
from pgmpy.inference import VariableElimination
from bamt.networks.discrete_bn import DiscreteBN
from bamt_inherited.networks.discrete import DiscreteBNGI
import shap
from sklearn import metrics as m
from tqdm import tqdm
from itertools import product
import optuna
from copy import copy

In [3]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [4]:
%run ../common_code.ipynb

In [5]:
#https://www.kaggle.com/datasets/camnugent/california-housing-prices
data = pd.read_csv("../../datasets/house_16H.csv")

n = data.shape[0]
target = "price"
features = ['P1', 'P5p1', 'P6p2', 'P11p4', 'P14p9', 'P15p1', 'P15p3', 'P16p2',
       'P18p2', 'P27p4', 'H2p2', 'H8p2', 'H10p1', 'H13p1', 'H18pA', 'H40p4'][:8]

data = data.rename(columns={feat: f'f{i}{feat}' for i, feat in enumerate(features)})
features = [f'f{i}{feat}' for i, feat in enumerate(features)]
data = data[features+[target]]
data['f0P1']=data['f0P1'].apply(lambda x: (x-data['f0P1'].min())/(data['f0P1'].max()-data['f0P1'].min()))
data[target]=data[target].apply(lambda x: (x-data[target].min())/(data[target].max()-data[target].min()))
data.loc[data[target]==0, target] = 0.01
n_features = len(features)
data.head(10)

,f0P1,f1P5p1,f2P6p2,f3P11p4,f4P14p9,f5P15p1,f6P15p3,f7P16p2,price
0,0.002118,0.460869,0.049252,0.226470,0.149827,0.752837,0.010057,0.579729,0.261199
1,0.000211,0.470968,0.002581,0.137419,0.096341,0.862581,0.000000,0.695142,0.081000
2,0.000647,0.485341,0.000211,0.189412,0.135656,0.856992,0.000000,0.683584,0.057400
3,0.000064,0.498929,0.000000,0.100642,0.085470,0.907923,0.000000,0.780488,0.057000
4,0.000042,0.474193,0.680645,0.225806,0.128834,0.896774,0.000000,0.756302,0.048200
5,0.000063,0.475054,0.086768,0.130152,0.095041,0.924078,0.000000,0.792683,0.029998
6,0.000098,0.506224,0.000000,0.121715,0.109244,0.865837,0.000000,0.724138,0.066400
7,0.010341,0.468334,0.291213,0.169686,0.123913,0.795254,0.009719,0.614848,0.132600
8,0.000402,0.467955,0.004408,0.180400,0.135755,0.828416,0.012886,0.674938,0.275999
9,0.000613,0.509800,0.007350,0.051893,0.035438,0.950779,0.003118,0.885986,0.281199


In [6]:
data.describe()

,f0P1,f1P5p1,f2P6p2,f3P11p4,f4P14p9,f5P15p1,f6P15p3,f7P16p2,price
count,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000,22784.000000
mean,0.001066,0.482297,0.063982,0.163900,0.121538,0.851076,0.020487,0.716131,0.100172
std,0.008996,0.031876,0.150921,0.080461,0.056687,0.079455,0.057666,0.087264,0.105666
min,0.000000,0.125000,0.000000,0.000000,0.000000,0.054156,0.000000,0.233702,0.010000
25%,0.000058,0.464647,0.000000,0.112069,0.081236,0.819686,0.000000,0.662283,0.042000
50%,0.000183,0.480396,0.003413,0.155485,0.117127,0.863645,0.000000,0.714286,0.066400
75%,0.000617,0.496040,0.033376,0.203596,0.156757,0.899657,0.019196,0.771039,0.112200
max,1.000000,0.923077,1.000000,0.917255,0.511872,1.000000,0.943325,1.000000,1.000000


In [98]:
data_corr = add_extra_features(data, features, extra_size=0.75, mode='prod')

In [99]:
data_dict_corr = {}

data_tv, data_dict_corr['test'] = train_test_split(data_corr, test_size=0.15, random_state=42)

data_dict_corr['train'], data_dict_corr['val'] = train_test_split(data_tv, test_size=0.15, random_state=42)

features_corr = list(data_corr.columns.difference([target]))

In [100]:
features_corr

['extra_001',
 'extra_002',
 'extra_003',
 'extra_004',
 'extra_005',
 'extra_006',
 'extra_007',
 'extra_008',
 'extra_009',
 'extra_010',
 'extra_011',
 'extra_012',
 'extra_013',
 'extra_014',
 'extra_015',
 'extra_016',
 'extra_017',
 'extra_018',
 'extra_019',
 'extra_020',
 'extra_021',
 'extra_022',
 'extra_023',
 'extra_024',
 'f0P1',
 'f1P5p1',
 'f2P6p2',
 'f3P11p4',
 'f4P14p9',
 'f5P15p1',
 'f6P15p3',
 'f7P16p2']

In [101]:
def objective(trial, features, target, data_dict):
    n_estimators= trial.suggest_int("n_estimators", low=params_net['n_estimators'][0], high=params_net['n_estimators'][1], step=1)
    max_depth= trial.suggest_int("max_depth", low=params_net['max_depth'][0], high=params_net['max_depth'][1], step=1)
    learning_rate=trial.suggest_float("learning_rate", low=params_net['learning_rate'][0], high=params_net['learning_rate'][1],log=True)
    
    regr = GradientBoostingRegressor(random_state=42, n_estimators=n_estimators,
                                     max_depth=max_depth, learning_rate=learning_rate)
    regr.fit(data_dict['train'][features], data_dict['train'][target])
    return score_regr(regr, data_dict['val'][features], data_dict['val'][target])

params_net = {
    'n_estimators': [10, 100],
    'max_depth': [3, 10],
    'learning_rate':[1e-5, 1]
}


# Initial model for error calculation

In [102]:
best_base_params = optunize_model(objective, features_corr, target, data_dict_corr, n_trials=20)

  0%|          | 0/20 [00:00<?, ?it/s]

In [103]:
best_base_params

{'n_estimators': 88, 'max_depth': 6, 'learning_rate': 0.12985501348561462}

In [104]:
model_corr = GradientBoostingRegressor(random_state=42, **best_base_params)
#model = LogisticRegression()
model_corr.fit(data_dict_corr['train'][features_corr], data_dict_corr['train'][target])

data_errors_corr = measure_error(data_dict_corr['val'], model_corr, features_corr, target, error_col_name='ape_error', task='regr')

In [105]:
n_features_corr = len(features_corr)
disc_data_corr, pipeline_data_corr = discretize(data_errors_corr, 
                                      features_corr+["ape_error"], icat=None, icont=list(range(n_features_corr+1)))

categories_corr = extract_categories(disc_data_corr)
encoder = OneHotEncoder(sparse=False)
disc_data_encoded_corr = pd.DataFrame(encoder.fit_transform(X=disc_data_corr), columns=categories_corr, dtype='uint8')
disc_data_encoded_corr.head(1)

,f0P1_0,f0P1_1,f0P1_2,f1P5p1_0,f1P5p1_1,f1P5p1_2,f2P6p2_1,f2P6p2_2,f3P11p4_0,f3P11p4_1,...,extra_022_1,extra_022_2,extra_023_0,extra_023_1,extra_023_2,extra_024_1,extra_024_2,ape_error_0,ape_error_1,ape_error_2
0,0,1,0,1,0,0,1,0,0,1,...,0,1,1,0,0,0,1,1,0,0


In [106]:
disc_data_corr['ape_error'].value_counts()

ape_error
1    1161
0     872
2     872
Name: count, dtype: int64

In [107]:
feat_to_grad_corr = {}
for feat in features_corr+['ape_error']:
    feat_to_grad_corr[feat]=[grad for grad in disc_data_encoded_corr.columns if feat in grad]


metric_dict = {
    'mape': lambda x, y: m.mean_absolute_percentage_error(x,y)*100,
    'r2_score': lambda x, y: m.r2_score(x, y)*100
}

gbn_fs = DiscreteBNGI('')

gbn_training_pipeline(gbn_fs, disc_data_encoded_corr, 'ape_error', categories_corr, score=BicScore)

2025-05-06 16:48:01,489 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-06 16:48:01,498 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-06 16:48:01,500 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-06 16:48:01,503 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-06 16:48:01,505 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-06 16:48:01,507 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-06 16:48:01,510 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-06 16:48:01,511 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-06 16:48:01,512 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-06 16:48:01,513 | ERROR    | GraphUtil

  0%|          | 0/200 [00:00<?, ?it/s]

In [110]:
gbn_fs.get_info()

,name,node_type,data_type,parents,parents_types
0,extra_007_2,Discrete,disc,[],[]
1,f4P14p9_2,Discrete,disc,[extra_007_2],[disc]
2,extra_003_2,Discrete,disc,[f4P14p9_2],[disc]
3,extra_007_0,Discrete,disc,"[f4P14p9_2, extra_003_2]","[disc, disc]"
4,ape_error_1,Discrete,disc,[extra_003_2],[disc]
...,...,...,...,...,...
79,extra_016_2,Discrete,disc,"[f2P6p2_2, f4P14p9_0, extra_017_2]","[disc, disc, disc]"
80,extra_006_1,Discrete,disc,"[f3P11p4_1, f7P16p2_0, extra_002_1, extra_022_2]","[disc, disc, disc, disc]"
81,extra_017_1,Discrete,disc,[extra_013_2],[disc]
82,extra_011_2,Discrete,disc,"[extra_005_2, extra_016_2]","[disc, disc]"


In [111]:
_df=gbn_fs.get_info()
_df[_df['name'].apply(lambda x: 'ape_error' in x.name)]

,name,node_type,data_type,parents,parents_types
4,ape_error_1,Discrete,disc,[extra_003_2],[disc]
27,ape_error_0,Discrete,disc,"[f4P14p9_2, f5P15p1_0]","[disc, disc]"
35,ape_error_2,Discrete,disc,"[f3P11p4_2, extra_003_2]","[disc, disc]"


# Univariate Feature selector

In [112]:
univ_sel = SelectKBest(f_regression, k=len(features_corr))
univ_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_univ = pd.DataFrame({'Feature': univ_sel.feature_names_in_, 'Metric value': univ_sel.pvalues_})

# RFE on decision tree

In [113]:
rfe_sel = RFE(estimator=DecisionTreeRegressor(random_state=42, max_depth=10,), n_features_to_select=1)
#rfe_sel = RFE(estimator=RandomForestClassifier(random_state=42, max_depth=10, n_estimators=50), n_features_to_select=1, )
rfe_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_rfe = pd.DataFrame({'Feature': rfe_sel.feature_names_in_, 'Metric value': rfe_sel.ranking_})

# Lasso from Logistic Regression with L1 penalty

In [119]:
lsvc = Lasso(alpha=0.00005, random_state=42)
lsvc.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
lasso_selector = SelectFromModel(lsvc, prefit=True)

In [120]:
lasso_support = [x for i, x in enumerate(features_corr) if lasso_selector.get_support()[i]]
lasso_support

['extra_003',
 'extra_005',
 'extra_006',
 'extra_010',
 'extra_020',
 'f3P11p4',
 'f5P15p1',
 'f6P15p3',
 'f7P16p2']

In [121]:
len(lasso_support)

9

## GBN selector

In [122]:
df_val_corr_bn=error_based_importance_features_bn(gbn_fs, 'ape_error', features_corr, feat_to_grad_corr).fillna(0)

100%|██████████| 32/32 [00:23<00:00,  1.39it/s]


## GBN selector run

In [123]:
gbn_res = elimination_experiment(model_corr, data_dict_corr, metric_dict, df_val_corr_bn, direction='asc', task='regr')

  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [31:55<00:00, 59.84s/it]


In [124]:
gbn_res.head()

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,32,-,51.413,38.22,0,0
1,31,extra_018,51.06,38.553,-0.686,0.87
2,30,extra_001,51.168,38.147,-0.477,-0.192
3,29,f6P15p3,51.028,38.888,-0.749,1.748
4,28,extra_013,51.014,38.81,-0.776,1.544


In [125]:
get_best_record(gbn_res, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
10,22,f0P1,50.723,39.176,-1.342,2.502


In [126]:
get_best_record(gbn_res, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
8,24,extra_004,51.397,39.588,-0.03,3.578


In [127]:
df_corr_excluded_asc_univ=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_univ, 'asc', task='regr')
df_corr_excluded_asc_univ.head()

100%|██████████| 32/32 [24:38<00:00, 46.19s/it]


,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,32,-,50.929,38.489,0,0
1,31,extra_003,51.173,38.577,0.479,0.228
2,30,f4P14p9,51.143,38.47,0.419,-0.048
3,29,extra_007,52.139,37.441,2.377,-2.723
4,28,extra_022,53.117,35.105,4.297,-8.791


In [128]:
get_best_record(df_corr_excluded_asc_univ, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,32,-,50.929,38.489,0,0


In [129]:
get_best_record(df_corr_excluded_asc_univ, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
1,31,extra_003,51.173,38.577,0.479,0.228


In [130]:
df_corr_excluded_rfe=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_rfe, 'desc', task='regr')
df_corr_excluded_rfe.head()

  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [21:44<00:00, 40.77s/it]


,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,32,-,51.362,38.131,0,0
1,31,extra_024,51.591,37.563,0.445,-1.489
2,30,extra_009,51.143,38.795,-0.426,1.742
3,29,extra_004,51.173,38.514,-0.368,1.004
4,28,f6P15p3,51.37,38.045,0.015,-0.226


In [131]:
get_best_record(df_corr_excluded_rfe, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
25,7,extra_010,50.184,37.522,-2.293,-1.597


In [132]:
get_best_record(df_corr_excluded_rfe, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
11,21,extra_020,51.247,39.311,-0.223,3.095


In [133]:
single_run(model_corr, data_dict_corr, lasso_support, target, metric_dict, task='regr')

[56.183, 34.581]

In [ ]:
#logging mape (probably woth to be negative mape)
0.5 random, base 18.467, gbn 17.877, univ 18.467, rfe 18.103 , lasso: 17.89
0.75 random, base 19.137, gbn 18.346, univ 19.137, rfe 18.239 , lasso: 18.338
0.5 noisy, base 18.337, gbn 17.927, univ 18.337, rfe  18.109, lasso: 18.221
0.75 noisy, base 18.442, gbn 17.960, univ 18.442, rfe  18.185, lasso: 18.586
0.5 prod, base 17.865, gbn 17.865, univ 17.865, rfe 17.802, lasso: 17.762
0.75 prod, base 18.442, gbn 18.417, univ 18.382, rfe 18.029, lasso: 18.585


In [ ]:
# logging r2_score
0.5 random, base 81.675, gbn 82.667, univ 81.675, rfe  82.27, lasso: 82.616 
0.75 random, base 80.482, gbn 81.832, univ 80.619, rfe  82.188, lasso: 81.806
0.5 noisy, base 81.813, gbn 82.479, univ 81.863, rfe  82.047, lasso: 81.88
0.75 noisy, base 81.637, gbn 82.039, univ 81.596, rfe  81.97, lasso: 81.392
0.5 prod, base 81.97, gbn 81.97, univ 82.176, rfe 82.438, lasso: 82.363
0.75 prod, base 81.637, gbn 81.637, univ 81.704, rfe 82.364, lasso: 80.941


In [ ]:
# #logging mape
# 0.5 random, base 17.877, gbn 80.075, univ 79.953, rfe 79.953, lasso: 79.944
# 0.75 random, base 80.119, gbn 80.55	, univ 80.119, rfe 80.119, lasso: 80.242

# 0.5 noisy base 80.42, gbn 80.58	, univ 80.42, rfe 80.578, lasso: 80.515
# 0.75 noisy base 80.264, gbn 80.311, univ 80.264, rfe 80.264, lasso: 80.633

# 0.5 prod base 80.007, gbn 80.203	, univ 80.022, rfe 80.448, lasso: 80.422
# 0.75 prod base 80.36, gbn 80.422	, univ 80.549, rfe 80.542, lasso: 80.316
